# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# load csv to dataframe
city_data = pd.read_csv("../WeatherPy/output_data/cities.csv")
city_data.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country
0,rikitea,-23.12,-134.97,68.20,59,87,5.78,PF
1,barrow,71.29,-156.79,35.60,96,90,3.60,US
2,georgetown,5.41,100.34,78.80,88,40,1.00,MY
3,atuona,-9.80,-139.03,79.59,77,17,7.77,PF
4,cape town,-33.93,18.42,66.20,55,0,2.60,ZA


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# configure gmaps
gmaps.configure(api_key=g_key)

# Lat and Lng as locations and humidity as weight
locations = city_data[["Lat", "Lng"]]
humidity = city_data["Humidity"]

In [4]:
# add heatmap layer to map and display 
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, 
                                 max_intensity=100, point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# narrow cities to ideal weather conditions
index = city_data[(city_data["Humidity"] > 45) | \
                  (city_data["Wind Speed"] > 5) | \
                  (city_data["Max Temp"] < 70) | \
                  (city_data["Max Temp"] > 80) | \
                  (city_data["Cloudiness"] > 20)].index.tolist()
hotel_df = city_data.drop(index)

# drop null values and change City to title case
hotel_df = hotel_df.dropna()
hotel_df["City"] = hotel_df["City"].str.title()
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country
109,Chitral,35.85,71.79,77.85,32,6,1.19,PK
183,Winnemucca,40.97,-117.74,77.00,22,1,3.60,US
281,Golden,39.76,-105.22,79.00,41,20,2.07,US
393,Wadersloh,51.73,8.25,72.00,42,5,2.60,DE
471,Lukovetskiy,64.30,41.92,77.00,33,0,3.00,RU
494,Maun,-19.98,23.42,73.40,25,0,4.10,BW


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# add column to dataframe
hotel_df["Hotel Name"] = ""

In [7]:
# set parameters to search for hotels within 5 km
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "keyword": "Hotel",
    "radius": 5000,
    "type": "hotel",
    "key": g_key
}

# use nearby search to find first hotel near city and store into dataframe
for index, row in hotel_df.iterrows():
    params["location"] = f"{row['Lat']},{row['Lng']}"
    response = requests.get(base_url, params=params).json()
    results = response["results"]
    try:
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
    except (KeyError, IndexError):
        print(f"There is no hotel within 5 km of {row['City']}.")

There is no hotel within 5 km of Wadersloh.


In [8]:
# display results
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Hotel Name
109,Chitral,35.85,71.79,77.85,32,6,1.19,PK,Al Mughal Hotel
183,Winnemucca,40.97,-117.74,77.00,22,1,3.60,US,Candlewood Suites Winnemucca
281,Golden,39.76,-105.22,79.00,41,20,2.07,US,Denver Marriott West
393,Wadersloh,51.73,8.25,72.00,42,5,2.60,DE,
471,Lukovetskiy,64.30,41.92,77.00,33,0,3.00,RU,Gostinitsa
494,Maun,-19.98,23.42,73.40,25,0,4.10,BW,Cresta Maun Hotel


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in \
              hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer on top of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))